# Chat completions with LumiOpen/Poro-34B-chat model 

Now we are ready to test chat-tuned [`LumiOpen/Poro-34B-chat`](https://huggingface.co/LumiOpen/Poro-34B-chat). We will start with a simple examples and move on to tweaking parameters that affect the output generated by the model. 

Poro-34B-Chat excels at:
* Answering questions in Finnish and English
* Following instructions in plain language
* Generating high-quality translations
* Writing code and assisting with development tasks

Let's start with basics, generate API key for this specific model and configure Aitta client to load the model `LumiOpen/Poro-34B-chat`. Since aitta-client is **OpenAI compatible** we can configure OpenAI client to use the OpenAI compatibility endpoints. We need to adjust the parameters `api_key` and ` base_url` when configuring the OpenAI client.

In [ ]:
# Set your personal model-specific API key here  
api_key = "<api-key>"

In [ ]:
from aitta_client import Model, Client, StaticAccessTokenSource
import openai


# configure Client instance with API URL and access token
token_source = StaticAccessTokenSource(api_key)
aitta_client = Client("https://api-staging-aitta.2.rahtiapp.fi", token_source)

# load the LumiOpen/Poro-34B-chat model
model = Model.load("LumiOpen/Poro-34B-chat", aitta_client)
print(model.description)

# configure OpenAI client to use the Aitta OpenAI compatibility endpoints
client = openai.OpenAI(api_key=token_source.get_access_token(), base_url=model.openai_api_url)


# Create chat completions

You must be eager to get started with actually getting responses from model based on your prompt.
Let's create one using  `client.chat.completions.create()` method and then explore how to fine-tune its parameters.

Here again diagram of the first API call using client for chat completions: 

![api-call-diagram-for-chat-completions](./images/API-chat-completions-bold.png)

In [ ]:
# perform chat completion with the OpenAI client
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "What is an API? Explain like I would be five."
        }
    ],
    model=model.id,
    stream=False  # response streaming is currently not supported by Aitta, now you get the full response in one go
)

print(chat_completion.choices[0].message.content)

# Understanding required parameters for `client.chat.completions.create` method

We just used `client.chat.completions.create` method to generate chat-based completions. This method requires two key parameters: `messages` and `model`. 

The model refers to the specific AI model that you’ve loaded with the Aitta client. The messages parameter is a list of message objects that represents the conversation history up to the point of the request. Each message object includes a `role` and `content`. The role determines how the model interprets the message, and it can be one of three types: `user`, `system`, or `assistant`. 
The `content` field is the actual text of the message. Understanding the role helps the model generate more accurate and contextually appropriate responses based on the conversation flow.

| Role | Description | Usage example |
|----|----|---|
| user | Represents the input from the user, such as a question or prompt. This helps initiate the conversation or provide queries. | `{ "role": "user", "content": "How does AI work?" }` |
| system | Provides guidelines or instructions for how the model should behave throughout the conversation. This is useful for setting the tone or framing the conversation. | `{ "role": "system", "content": "You are an assistant who always provides concise answers." }` |
| assistant | Represents the AI's response to the user, guiding the conversation forward. This role helps the model generate replies based on the user's input. | `{ "role": "assistant", "content": "AI works by processing large amounts of data to make predictions or decisions." }` |

In [ ]:
# Example messages to prompt the model
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Can you explain what AI is?"},
    {"role": "assistant", "content": "AI stands for Artificial Intelligence. It refers to machines that can perform tasks that typically require human intelligence."},
    {"role": "user", "content": "How does AI work?"}
]

# Perform a chat completion request
chat_completion = client.chat.completions.create(
    messages=messages,
    model=model.id,
    stream=False
)

# Output the response
print(chat_completion.choices[0].message.content)


In [ ]:
# Uncomment below line to see documentation for this method
#help(client.chat.completions.create)

# Understanding optional parameters for `client.chat.completions.create` method

There are several parameters you can adjust to fine-tune the generated responses. These parameters help control the randomness, length, and number of responses generated. Below are the parameters supported for OpenAI compatibility at the moment. 


| Parameter | Description | Effect |
|-----------|-------------|--------|
| **top_p** (nucleus sampling) | A value between 0 and 1. `top_p` controls how the model picks the next word by considering only the most likely options. | Lower values (like 0.5) make the model more focused and predictable. Higher values (like 0.9) make the model more creative and varied. |
| **temperature** (randomness control) | A value between 0 and 2. `temperature` controls the creativity of the response. Higher values create more varied, creative responses, while lower values produce more focused, deterministic outputs. | A higher temperature increases creativity, while a lower temperature makes responses more predictable. |
| **max_completion_tokens** (maximum tokens)| Limits the number of tokens (words or parts of words) the model can generate in a response. | A smaller value shortens the response, while a larger value allows for more detailed answers. |
| **n** (number of completions)| A higher value (e.g., 3) generates multiple variations of responses, offering different perspectives or creative options. |

You can explore how these affect responses generated by the model.  

**Note that longer input prompts and higher `max_completion_tokens` can increase the time it takes to receive a response.**


*Tip: Use `temperature` and `top_p` together cautiously. Typically, adjusting one is enough—setting both too high may lead to overly random responses, while both too low may make responses overly constrained.*


## Try it yourself

Now it's time for you to experiment with these parameters.

You may have noticed that sometimes responses are cut off mid-sentence. You can influence this by adjusting `max_completion_tokens`. However, even with a higher value, truncation might still occur.

In [ ]:
### CODE HERE ###